# Problemas de contorno

Los problemas antes resueltos consistían en problemas de valor inicial, el cual es el tipo más frecuente de problemas de EDOs encontrados en física pero no el único pues podemos encontrar también **problemas de contorno o boundary value problems**. 

Por ejemplo, la EDO que gobierna el movimiento de una pelota lanzada al aire en función de su altura $x$, es: $$\dfrac{d^2 x}{dt^2} = -g$$

Así como podemos especificar su posición y velocidad inicial, también podríamos indicar que tenía una altura $x=0$ en $t=0$ y que $x=0$ en $t=t_1$: es decir, especificamos el tiempo en que se lanzó al aire y el tiempo en que cayó y buscaremos la solución que satisface estas condiciones.

## The shooting method

Es un método de intento-error que busca los correctos valores de las condiciones iniciales que encajen con un cierto conjunto de condiciones de contorno. 

Por ejemplo, en el problema del lanzamiento de una pelota, tenemos la posición inicial de la pelota pero no su velocidad. Entonces, empezamos a estimar un valor para esta velocidad y resolvemos la EDO y calculamos su posición para un tiempo $t=t_1$. Si en este instante no alcanza $x=0$ entonces volvemos a estimar un nuevo valor y calculamos de nuevo.

<img src="Capt-Ch86.png" width="500px">

**¿Cómo encontramos esta velocidad?** Si suponemos que la posición de la pelota en un tiempo $t_1$ está dada por $x = f(v)$ para una cierta velocidad inicial, entonces para solucionar el problema, bastará con resolver $f(v) = 0$ y se determinará $v$. 

Entonces, usamos un método para resolver la EDO (ej: RK4) para calcular el valor de la función $f(v)$ y luego empleamos un método para buscar raíces (ej: brentq en scipy) para determinar $v$ que encaje con las condiciones de contorno.

### Posición vertical de una pelota lanzada

Para el problema antes descrito, supongamos que $x=0$ en $t=0$ y $x=0$ en $t=10$. El primer paso es convertir la EDO de segundo orden en dos de primer orden:

$$\dfrac{dx}{dt} = y, \quad \dfrac{dy}{dt} = -g$$

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq

In [2]:
g = 9.81       # Aceleración de la gravedad
a = 0.0        # Tiempo inicial   
b = 10.0       # Tiempo final  
N = 1000       # Número de pasos de RK4    
h = (b-a)/N    # Tamaño de paso de RK4
target = 1e-10 # Precisión para brentq

In [3]:
# Parte derecha de la EDO
def f(r):
    x, y = r
    fx, fy = y, -g
    return np.array([fx, fy], float)

# Resolución del sistema de EDOs
# Retorna la altura final
def height(v): 
    r = np.array([0.0, v], float)
    for t in np.arange(a, b, h):
        k1 = h*f(r)
        k2 = h*f(r + 0.5*k1)
        k3 = h*f(r + 0.5*k2)
        k4 = h*f(r + k3)
        r += (k1 + 2*k2 + 2*k3 + k4)/6
    return r[0]

v = brentq(height, 1, 100, xtol=target)
print('La velocidad inicial requerida es {0: .2f} m/s'.format(v))

La velocidad inicial requerida es  49.05 m/s


## The relaxation method

Definimos una forma para la solución completa que encaje con las condiciones de contorno (que puede no ser la correcta en la región entre los extremos. Luego, se modifica sucesivamente esta función para llevarla lo más cerca posible a la solución de la EDO, asegurándonos que satisface la ecuación de movimiento. Este es más usado para EDPs.

# Problemas de autovalores

Problemas de contorno para ecuaciones lineales homogéneas. Por ejemplo, la ecuación de Schrodinger independiente del tiempo para una partícula de masa $m$ en 1-D
$$-\dfrac{\hbar^2}{2m} \dfrac{d^2 \psi}{dx^2} + V(x) \psi(x) = E \psi(x),$$
donde $\psi(x)$ es la función de onda, $V(x)$ es el potencial y $E$ es la energía total de la partícula.

En específico, para una partícula en un potencial de paredes infinitas
$$V(x) = \begin{cases} 0, & \text{para} \ 0 < x < L \\
\infty, & \text{otros casos} \end{cases}$$

Luego
$$\dfrac{d\psi}{dx} = \phi$$

$$\dfrac{d\phi}{dx} = \dfrac{2m}{\hbar^2} [V(x) - E] \psi$$

Empleando el shooting method, sabemos que la condición inicial para $\psi$ es $\psi(x=0) = 0$, y para $\phi$ tendríamos que estimar un valor inicial tal que $\psi(x=L) = 0$. Sin embargo, esto no funcionará. 

La modificación del valor inicial de $\phi$ afecta la magnitud general de la solución y no su forma. Si duplicamos el valor inicial de $d\psi/dx$ obtendremos $2\psi(x)$ como solución de la ecuación, es decir, no podremos alcanzar cero en $x=L$ como se ve en el dibujo si $\psi(x)$ no pasa por cero en $x=L$.

<img src="Capt-Ch863.png" width="500px">

En este caso, se tienen soluciones de la ecuación relacionadas a ciertos valores del parámetro de energía E (autovalores). Estos valores son discretos y constituye una mejor opción variar esta parámetro en vez del valor inicial de $d\psi/dx$.

Entonces, podemos pensar la solución de la ecuación de Schrodinger como $f(E)$ igual al valor de la autofunción en $x=L$ y buscar el valor de $E$ que anula esta función. En este sentido, el valor de $d\psi/dx$ es irrelevante.

### Ground state energy in a square well

In [22]:
# Constantes
m = 9.1094e-31 # Masa del electrón
ħ = 1.0546e-34 # hbar
e = 1.6022e-19 # Carga del electrón
L = 5.2918e-11 # Radio de Bohr
N = 1000
h = L/N

# Potencial: puede modificarse
def V(x):
    return 0.0

def f(r, x, E):
    psi, phi = r
    fpsi = phi
    fphi = (2*m/ħ**2)*(V(x) - E)*psi
    return np.array([fpsi, fphi], float)

# Cálculo de la función de onda para una energía E
def solve(E):
    psi = 0.0
    phi = 1.0
    r = np.array([psi, phi], float)
    
    for x in np.arange(0, L, h):
        k1 = h*f(r, x, E)
        k2 = h*f(r + 0.5*k1, x + 0.5*h, E)
        k3 = h*f(r + 0.5*k2, x + 0.5*h, E) 
        k4 = h*f(r + k3, x + h, E)
        r += (k1 + 2*k2 + 2*k3 + k4)/6
        
    return r[0]

# Encontrar la energía
E = brentq(solve, 1*e, 149*e, xtol=e/1000)
print('La energía del estado fundamental es {0: .2f} eV'.format(E/e))

La energía del estado fundamental es  134.29 eV
